In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applied options are for CSV files.
# For other file types, these will be ignored.
df = spark.read.csv(
  file_location,
  header=True,
  inferSchema=True
)

df.show(5)

+----------+----+------+------+---+------+----+
total_bill| tip| sex|smoker|day| time|size|
+----------+----+------+------+---+------+----+
 16.99|1.01|Female| No|Sun|Dinner| 2|
 10.34|1.66| Male| No|Sun|Dinner| 3|
 21.01| 3.5| Male| No|Sun|Dinner| 3|
 23.68|3.31| Male| No|Sun|Dinner| 2|
 24.59|3.61|Female| No|Sun|Dinner| 4|
+----------+----+------+------+---+------+----+
only showing top 5 rows

In [0]:
df.printSchema()

root
-- total_bill: double (nullable = true)
-- tip: double (nullable = true)
-- sex: string (nullable = true)
-- smoker: string (nullable = true)
-- day: string (nullable = true)
-- time: string (nullable = true)
-- size: integer (nullable = true)

In [0]:
df.columns

Out[3]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
# Handle categorical features
# StringIndexer <--> OneHot Encoder (sklearn)
from pyspark.ml.feature import StringIndexer

In [0]:
# Creating indexer object for 1 categorical col
indexer = StringIndexer(
  inputCol="sex",
  outputCol="sex_indexed"
)

df_tmp = indexer.fit(df).transform(df)

df_tmp.show(3)

+----------+----+------+------+---+------+----+-----------+
total_bill| tip| sex|smoker|day| time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
 16.99|1.01|Female| No|Sun|Dinner| 2| 1.0|
 10.34|1.66| Male| No|Sun|Dinner| 3| 0.0|
 21.01| 3.5| Male| No|Sun|Dinner| 3| 0.0|
+----------+----+------+------+---+------+----+-----------+
only showing top 3 rows

In [0]:
# Creating indexer object for 2 or more categorical cols
indexer = StringIndexer(
  inputCols=["sex", "smoker", "day", "time"],
  outputCols=["sex_indexed", "smoker_indexed", "day_indexed", "time_indexed"]
)

df_tmp = indexer.fit(df).transform(df)

df_tmp.show(3)

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
total_bill| tip| sex|smoker|day| time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
 16.99|1.01|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|
 10.34|1.66| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
 21.01| 3.5| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
only showing top 3 rows

In [0]:
# For PySpark, we have to group independent features together and the dependent feature separatively with an assembler vector
# It is needed before running a model
# thx to it, all datas from all clusters will be processed in the same way
from pyspark.ml.feature import VectorAssembler

# input cols
list_input_cols = [x if x in ["tip", "size"] else x + "_indexed" for x in df.columns[1:]]

# Initializaton
# inputCols -> independent features
# outputCol -> independent features group name
# outputCol is a specific channel created in PySpark
feature_assembler = VectorAssembler(
  inputCols=list_input_cols,
  outputCol="Independent_Features"
)

# fit and transform
output = feature_assembler.transform(df_tmp)

In [0]:
output.select("Independent_Features").show(3)

+--------------------+
Independent_Features|
+--------------------+
[1.01,1.0,0.0,1.0...|
[1.66,0.0,0.0,1.0...|
[3.5,0.0,0.0,1.0,...|
+--------------------+
only showing top 3 rows

In [0]:
finalized_data = output.select(["Independent_Features", "total_bill"])
finalized_data.show(3)

+--------------------+----------+
Independent_Features|total_bill|
+--------------------+----------+
[1.01,1.0,0.0,1.0...| 16.99|
[1.66,0.0,0.0,1.0...| 10.34|
[3.5,0.0,0.0,1.0,...| 21.01|
+--------------------+----------+
only showing top 3 rows

In [0]:
# shape function
def spark_shape(self):
  return (self.count(), len(self.columns))

In [0]:
# Train Test Split
train_data, test_data = finalized_data.randomSplit(
  [0.75, 0.25],
  seed=0
)

print(f"train shape = {spark_shape(train_data)}")
print(f"test shape  = {spark_shape(test_data)}")

train shape = (179, 2)
test shape = (65, 2)

In [0]:
# Linear regression
from pyspark.ml.regression import LinearRegression

linReg_model = LinearRegression(
  featuresCol="Independent_Features",
  labelCol="total_bill",
  elasticNetParam=0,
  standardization=True
)

linReg_model = linReg_model.fit(train_data)

In [0]:
# Print the coefficients and intercept for linear regression
print(f"Coefficients: {linReg_model.coefficients}")
print(f"Intercept: {linReg_model.intercept}")

# Prediction
pred_results = linReg_model.evaluate(test_data)

# Metrics
print(f"\nRMSE: {pred_results.rootMeanSquaredError}")
print(f"MAE: {pred_results.meanAbsoluteError}")
print(f"r2: {pred_results.r2}")

Coefficients: [2.949308651974232,-1.8347251777297107,3.0105186503007557,0.2781009627562797,-1.2523549229722268,3.5331915525544817]
Intercept: 1.4613639822213287

RMSE: 6.088241527460466
MAE: 4.612374619672759
r2: 0.6043445005735291

In [0]:
pred_results.predictions.show(5)

+--------------------+----------+------------------+
Independent_Features|total_bill| prediction|
+--------------------+----------+------------------+
(6,[0,5],[1.25,2.0])| 10.07|12.214382902298082|
(6,[0,5],[3.27,2.0])| 17.78|18.171986379286032|
(6,[0,5],[3.35,3.0])| 20.65|21.941122623998453|
(6,[0,5],[5.92,3.0])| 29.03| 29.52084585957223|
(6,[0,5],[6.73,4.0])| 48.27|35.442977420225844|
+--------------------+----------+------------------+
only showing top 5 rows